# Обнаружение выбросов по углам (Angle-Based Outlier Detection)
![Обнаружение выбросов по углам](abod.jpg)
### Данные
* https://video.ittensive.com/machine-learning/hacktherealty/exposition_train.basic.csv.gz

### Подключение библиотек

In [1]:
import pandas as pd
import numpy as np
from sklearn.cluster import KMeans
from pyod.models.abod import ABOD
from sklearn import metrics
from sklearn.preprocessing import StandardScaler

### Загрузка данных

In [2]:
train_data = pd.read_csv('https://video.ittensive.com/machine-learning/hacktherealty/exposition_train.basic.csv.gz')
train_data.head()

,total_area,ceiling_height,rooms,living_area,price,day_mean,doy_108,price_locality_name_median,target
0,105.000000,3.00,3,50.000000,95000,2.456912,0,2.261905,1
1,40.000000,3.00,1,19.200001,25000,3.028689,0,1.000000,2
2,37.599998,2.64,0,19.000000,26000,3.091993,0,0.619048,2
3,80.000000,3.00,3,49.000000,35000,3.101010,0,1.250000,2
4,100.000000,3.00,3,49.000000,80000,2.495468,0,1.904762,3


### Нормализация данных
Приведение всех значений к отрезку [0;1], удалим из данных target

In [3]:
train_data_an = pd.DataFrame(StandardScaler().fit_transform(train_data[train_data.columns[:-1]]))

In [4]:
train_data_an.head()

,0,1,2,3,4,5,6,7
0,1.709793,1.115052,1.469889,1.025395,0.378181,-1.607090,-0.108831,0.278301
1,-0.483202,1.115052,-0.861151,-0.679471,-0.217544,0.337784,-0.108831,-0.132179
2,-0.564174,-0.645106,-2.026671,-0.690542,-0.209033,0.553112,-0.108831,-0.256098
3,0.866333,1.115052,1.469889,0.970042,-0.132440,0.583783,-0.108831,-0.050858
4,1.541101,1.115052,1.469889,0.970042,0.250526,-1.475943,-0.108831,0.162128


### ABOD

In [5]:
abod = ABOD(contamination=0.001).fit(train_data_an)

c:\users\lenovo pc\appdata\local\programs\python\python38\lib\site-packages\numpy\core\fromnumeric.py:3702: RuntimeWarning: Degrees of freedom <= 0 for slice
  return _methods._var(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
c:\users\lenovo pc\appdata\local\programs\python\python38\lib\site-packages\numpy\core\_methods.py:221: RuntimeWarning: invalid value encountered in true_divide
  arrmean = um.true_divide(arrmean, div, out=arrmean, casting='unsafe',
c:\users\lenovo pc\appdata\local\programs\python\python38\lib\site-packages\numpy\core\_methods.py:253: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


Отфильтруем аномальные данные

In [6]:
abod.labels_[np.where(abod.labels_==1)]

array([], dtype=int32)

In [7]:
train_data_filtered = train_data_an[train_data_an.index.isin(np.where(abod.labels_==0)[0])]
print ("Процент аномалий:", round(100*len(abod.labels_[np.where(abod.labels_==1)])/len(train_data_an),2), "%")

Процент аномалий: 0.0 %


In [8]:
train_data_filtered.head()

,0,1,2,3,4,5,6,7
0,1.709793,1.115052,1.469889,1.025395,0.378181,-1.607090,-0.108831,0.278301
1,-0.483202,1.115052,-0.861151,-0.679471,-0.217544,0.337784,-0.108831,-0.132179
2,-0.564174,-0.645106,-2.026671,-0.690542,-0.209033,0.553112,-0.108831,-0.256098
3,0.866333,1.115052,1.469889,0.970042,-0.132440,0.583783,-0.108831,-0.050858
4,1.541101,1.115052,1.469889,0.970042,0.250526,-1.475943,-0.108831,0.162128


### Сравним K-средних
Построим модель на отфильтрованных данных

In [9]:
kmeans_an = KMeans(n_clusters=100, random_state=0, max_iter=100, n_init=10).fit(train_data_filtered)

In [10]:
train_data_an["target_cluster_an"] = kmeans_an.predict(train_data_an)

### Оценка точности предсказания
Присвоим среднее значение по кластеру. Без фильтрации: 4.18, Смирнов: 4.16, Эллипсоидальная аппроксимация/LOF: 4.19

In [11]:
train_data_an["target"] = train_data["target"]

In [12]:
target_cluster_an = np.round(train_data_an.groupby("target_cluster_an").mean()["target"])
train_data_an["target_pred_an"] = train_data_an["target_cluster_an"].apply(lambda x: target_cluster_an[x])

In [13]:
print ("Фильтрация аномалий: ", np.exp(np.abs(train_data_an["target"] - train_data_an["target_pred_an"])).sum() / len(train_data_an))

Фильтрация аномалий:  4.186247152543807
